In [1]:
from fomo.models.clip.clip_base import ClipClassifier
import torch
from PIL import Image
import numpy as np
from clip import clip as og_clip

/Users/dominykas.seputis/github/fomo/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clip = ClipClassifier()
clip.to_cpu()
clip.eval();

In [3]:
clip.precompute_prompt_features(["a picture of a cat", "a picture of kitten"])

In [4]:
img = Image.open("./cat.jpg")

In [5]:
img_tensor = clip.transform(img).view(1, 3, 224, 224)

In [7]:
with torch.no_grad():
    logits = clip.forward(img_tensor)
print(logits)

tensor([[29.0192, 26.0525]])


In [8]:
torch.functional.F.softmax(logits, dim=1)

tensor([[0.9510, 0.0490]])

In [8]:
with torch.no_grad():
 a = clip.encode_images(img)

In [9]:
clip._precomputed_prompt_features.shape

torch.Size([2, 512])

In [10]:
logit_scale = clip.logit_scale
image_features = a
text_features = clip._precomputed_prompt_features

In [11]:
o = logit_scale * image_features @ text_features.t()
o -= o.max()
o = torch.exp(o)
o /= o.sum()
o

tensor([[0.9688, 0.0316]], dtype=torch.float16)

In [12]:
torch.exp(o - o.max())

tensor([[1.0000, 0.3918]], dtype=torch.float16)

In [13]:
with torch.no_grad():
    clip.forward(img)